In [ ]:
pip install torch torchvision

In [ ]:
pip install pytorch-pretrained-bert

In [1]:
# Dependencies
import pandas as pd 
import numpy as np 
import torch.nn as nn
from pytorch_pretrained_bert import BertTokenizer, BertModel
import torch
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [13]:
# Read in csv file
pd.set_option('display.max_columns', 30)
fake_jobs_df = pd.read_csv("fake_jobs_clean.csv")
fake_jobs_df

,job_id,city,state/province,country,title,department,industry,function,salary_range,salary_provided,company_profile,description,requirements,benefits,benefits_provided,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,fraudulent
0,1,New York,NY,US,Marketing Intern,Marketing,NaN,Marketing,NaN,0,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,0,1,0,Other,Internship,NaN,0
1,2,Auckland,,NZ,Customer Service - Cloud Video Production,Success,Marketing and Advertising,Customer Service,NaN,0,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,1,0,1,0,Full-time,Not Applicable,NaN,0
2,3,Wever,IA,US,Commissioning Machinery Assistant (CMA),NaN,NaN,NaN,NaN,0,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,0,1,0,NaN,NaN,NaN,0
3,4,Washington,DC,US,Account Executive - Washington DC,Sales,Computer Software,Sales,NaN,0,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,1,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,0
4,5,Fort Worth,FL,US,Bill Review Manager,NaN,Hospital & Health Care,Health Care Provider,NaN,0,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,1,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,17876,Toronto,ON,CA,Account Director - Distribution,Sales,Computer Software,Sales,NaN,0,Vend is looking for some awesome new talent to...,Just in case this is the first time you’ve vis...,To ace this role you:Will eat comprehensive St...,What can you expect from us?We have an open cu...,1,0,1,1,Full-time,Mid-Senior level,NaN,0
17876,17877,Philadelphia,PA,US,Payroll Accountant,Accounting,Internet,Accounting/Auditing,NaN,0,WebLinc is the e-commerce platform and service...,The Payroll Accountant will focus primarily on...,- B.A. or B.S. in Accounting- Desire to have f...,Health &amp; WellnessMedical planPrescription ...,1,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,0
17877,17878,Houston,TX,US,Project Cost Control Staff Engineer - Cost Con...,NaN,NaN,NaN,NaN,0,We Provide Full Time Permanent Positions for m...,Experienced Project Cost Control Staff Enginee...,At least 12 years professional experience.Abil...,NaN,0,0,0,0,Full-time,NaN,NaN,0
17878,17879,Lagos,LA,NG,Graphic Designer,NaN,Graphic Design,Design,NaN,0,NaN,Nemsia Studios is looking for an experienced v...,1. Must be fluent in the latest versions of Co...,Competitive salary (compensation will be based...,1,0,0,1,Contract,Not Applicable,Professional,0


In [14]:
# Take a look at job description and fraudulent cols
df = fake_jobs_df[['description', 'fraudulent']]
df

,description,fraudulent
0,"Food52, a fast-growing, James Beard Award-winn...",0
1,Organised - Focused - Vibrant - Awesome!Do you...,0
2,"Our client, located in Houston, is actively se...",0
3,THE COMPANY: ESRI – Environmental Systems Rese...,0
4,JOB TITLE: Itemization Review ManagerLOCATION:...,0
...,...,...
17875,Just in case this is the first time you’ve vis...,0
17876,The Payroll Accountant will focus primarily on...,0
17877,Experienced Project Cost Control Staff Enginee...,0
17878,Nemsia Studios is looking for an experienced v...,0


In [15]:
# Count how many Real vs Fraudulent postings 
from collections import Counter
print(Counter(df['fraudulent'].values))

Counter({0: 17014, 1: 866})


In [16]:
# Drop NaN values
df.dropna(inplace = True)
df

,description,fraudulent
0,"Food52, a fast-growing, James Beard Award-winn...",0
1,Organised - Focused - Vibrant - Awesome!Do you...,0
2,"Our client, located in Houston, is actively se...",0
3,THE COMPANY: ESRI – Environmental Systems Rese...,0
4,JOB TITLE: Itemization Review ManagerLOCATION:...,0
...,...,...
17875,Just in case this is the first time you’ve vis...,0
17876,The Payroll Accountant will focus primarily on...,0
17877,Experienced Project Cost Control Staff Enginee...,0
17878,Nemsia Studios is looking for an experienced v...,0


In [17]:
# Make sure number of real vs fraudulent are more balanced
# Using UNDERsampling for balancing of classification groups
df_fraud= df[df['fraudulent'] == 1] 
df_normal = df[df['fraudulent'] == 0] 

df_normal = df_normal.sample(n=len(df_fraud), random_state=22)
df = df_normal.append(df_fraud)

df

,description,fraudulent
9024,Training OfficerDistribution Channels Training...,0
1645,We are looking for a talented and experienced ...,0
2537,We are an established and progressive chemical...,0
9242,The key areas of responsibility in this positi...,0
12967,WDM Group is seeking a top level RELATIONSHIP ...,0
...,...,...
17827,Student Positions Part-Time and Full-Time.You ...,1
17828,LEARN TO EARN AN EXECUTIVE LEVEL INCOMEFULL TR...,1
17829,inFullMobile Sp. z o.o. is a mobile software d...,1
17830,JOB DESCRIPTIONWe are seeking a full time payr...,1


In [18]:
# Shuffle the dataframe so fraudulent postings more evenly distributed 
from sklearn.utils import shuffle
df = shuffle(df, random_state=22)
df = df.reset_index(drop=True)
df

,description,fraudulent
0,Giromax technology is currently looking for...,1
1,"Play with kids, get paid for it Love travel? J...",0
2,Applied Memetics LLC (AM LLC) is seeking a Rus...,0
3,Arise Virtual Solutions is a business process ...,1
4,"Pinnatta, οne of the fastest growing Greek sta...",0
...,...,...
1725,About Us:QB Medical Inc. is a medical supply d...,0
1726,Prepare yourself to learn about an exciting wa...,1
1727,Communicator. Writer. People-person.German-nat...,0
1728,Looking for a change? Not happy where you are...,0


In [19]:
# Count Real vs Fraudulent again to make sure we've achieved the desired result
print(Counter(df['fraudulent'].values))

Counter({1: 865, 0: 865})


In [20]:
# Split the data into training and testing
train_data = df.head(865)
test_data = df.tail(865)

In [21]:
# Generate a list of dictionaries with ‘description’ and ‘fraudulent’ keys
train_data = [{'description': description, 'fraudulent': fraudulent } for description in list(train_data['description']) for fraudulent in list(train_data['fraudulent'])]

test_data = [{'description': description, 'fraudulent': fraudulent } for description in list(test_data['description'])for fraudulent in list(test_data['fraudulent'])]


In [22]:
# Generate a list of tuples from the list of dictionaries
train_texts, train_labels = list(zip(*map(lambda d: (d['description'], d['fraudulent']), train_data)))
test_texts, test_labels = list(zip(*map(lambda d: (d['description'], d['fraudulent']), test_data)))

In [23]:
# Generate tokens and token ids
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:511], train_texts))
test_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:511], test_texts))

train_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, train_tokens))
test_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, test_tokens))

train_tokens_ids = pad_sequences(train_tokens_ids, maxlen=512, truncating="post", padding="post", dtype="int")
test_tokens_ids = pad_sequences(test_tokens_ids, maxlen=512, truncating="post", padding="post", dtype="int")

KeyboardInterrupt: 

In [ ]:
# Generate a boolean array based on the value of ‘fraudulent’ for our testing and training sets
train_y = np.array(train_labels) == 1
test_y = np.array(test_labels) == 1